<a href="https://colab.research.google.com/github/BinduParvati7/Capstone5588/blob/Bindu's-work/Group5_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing the Required libraries
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import math
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train = pd.read_csv("/content/drive/MyDrive/DS_Capstone/fraudTrain.csv", index_col=0)
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
test = pd.read_csv("/content/drive/MyDrive/DS_Capstone/fraudTest.csv", index_col=0)
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
merged_df = pd.concat([train, test], axis=0, ignore_index=True)

In [7]:
merged_df = merged_df.drop(['first','last','trans_num'], axis=1)

In [8]:
merged_df.shape

(1852394, 19)

In [9]:
#merged_df.to_csv('merged_data.csv', index=False)

In [10]:
merged_df.head(5)

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,city,state,zip,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,561 Perry Cove,Moravian Falls,NC,28654,36.08,-81.18,3495,"Psychologist, counselling",1988-03-09,1325376018,36.01,-82.05,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.89,-118.21,149,Special educational needs teacher,1978-06-21,1325376044,49.16,-118.19,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,594 White Dale Suite 530,Malad City,ID,83252,42.18,-112.26,4154,Nature conservation officer,1962-01-19,1325376051,43.15,-112.15,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.23,-112.11,1939,Patent attorney,1967-01-12,1325376076,47.03,-112.56,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,408 Bradley Rest,Doe Hill,VA,24433,38.42,-79.46,99,Dance movement psychotherapist,1986-03-28,1325376186,38.67,-78.63,0


In [11]:
merged_df['trans_date_trans_time'] = pd.to_datetime(merged_df['trans_date_trans_time'])
merged_df['dob'] = pd.to_datetime(merged_df['dob'])

In [12]:

# Extract relevant information from datetime columns
merged_df['trans_year'] = merged_df['trans_date_trans_time'].dt.year
merged_df['trans_month'] = merged_df['trans_date_trans_time'].dt.month
merged_df['trans_day'] = merged_df['trans_date_trans_time'].dt.day
merged_df['dob_year'] = merged_df['dob'].dt.year
merged_df['dob_month'] = merged_df['dob'].dt.month
merged_df['dob_day'] = merged_df['dob'].dt.day

# Drop the original datetime columns
merged_df = merged_df.drop(['trans_date_trans_time', 'dob'], axis=1)

In [13]:
merged_df.shape

(1852394, 23)

In [14]:
cleandf = merged_df.copy()
cleandf.shape

(1852394, 23)

#Dividing data into eastcost and west coast

In [15]:
# List of East Coast state codes
east_coast_states = ['ME', 'NH', 'VT', 'MA', 'RI', 'CT', 'NY', 'NJ', 'PA', 'DE', 'MD', 'VA', 'NC', 'SC', 'GA', 'FL', 'WV', 'OH', 'KY', 'TN', 'MS', 'AL', 'WI', 'IL']

# Filter the data for East Coast states
east_coast_df = cleandf[cleandf['state'].isin(east_coast_states)]

In [16]:
east_coast_df.shape

(973613, 23)

In [17]:
value_counts = east_coast_df['is_fraud'].value_counts()
print(value_counts)

0    968367
1      5246
Name: is_fraud, dtype: int64


In [18]:
west_coast_df = merged_df[~merged_df['state'].isin(east_coast_states)]
wc_df = west_coast_df.copy()

In [19]:

west_coast_df.shape

(878781, 23)

In [20]:
wc_df.shape

(878781, 23)

In [21]:
wc_df.shape

(878781, 23)

In [22]:
w_value_counts = wc_df['is_fraud'].value_counts()
print(w_value_counts)

0    874376
1      4405
Name: is_fraud, dtype: int64


In [23]:
# Preprocessing
# Assuming you have already converted 'trans_date_trans_time' and 'dob' columns to datetime
# Encode categorical columns using LabelEncoder
label_encoders = {}
categorical_columns = ['merchant', 'category', 'gender', 'street', 'city', 'state', 'job']
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    wc_df[col] = label_encoders[col].fit_transform(wc_df[col])

# Split the data into features (X) and target (y)
X = wc_df.drop(['is_fraud'], axis=1)
y = wc_df['is_fraud']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
wc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878781 entries, 1 to 1852393
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   cc_num       878781 non-null  int64  
 1   merchant     878781 non-null  int64  
 2   category     878781 non-null  int64  
 3   amt          878781 non-null  float64
 4   gender       878781 non-null  int64  
 5   street       878781 non-null  int64  
 6   city         878781 non-null  int64  
 7   state        878781 non-null  int64  
 8   zip          878781 non-null  int64  
 9   lat          878781 non-null  float64
 10  long         878781 non-null  float64
 11  city_pop     878781 non-null  int64  
 12  job          878781 non-null  int64  
 13  unix_time    878781 non-null  int64  
 14  merch_lat    878781 non-null  float64
 15  merch_long   878781 non-null  float64
 16  is_fraud     878781 non-null  int64  
 17  trans_year   878781 non-null  int64  
 18  trans_month  878781 non

**Sequential Model**

Input Layer: It takes an input with the shape of (X_train.shape[1],) which is the shape of your training data.

Dense Layer (64 units): It is a fully connected (dense) layer with 64 units and ReLU activation.

Dense Layer (32 units): Another fully connected layer with 32 units and ReLU activation.

Dense Layer (1 unit): The output layer with a single unit and sigmoid activation. This is a binary classification model, and the sigmoid activation function is often used for binary classification tasks.

After defining the model, you compile it using the Adam optimizer, binary cross-entropy loss (commonly used for binary classification problems), and accuracy as the metric for evaluation.

In [25]:

# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
17576/17576 [==============================] - 46s 3ms/step - loss: 0.0215 - accuracy: 0.9947 - val_loss: 0.0173 - val_accuracy: 0.9955
Epoch 2/5
17576/17576 [==============================] - 45s 3ms/step - loss: 0.0165 - accuracy: 0.9959 - val_loss: 0.0145 - val_accuracy: 0.9964
Epoch 3/5
17576/17576 [==============================] - 42s 2ms/step - loss: 0.0144 - accuracy: 0.9964 - val_loss: 0.0137 - val_accuracy: 0.9966
Epoch 4/5
17576/17576 [==============================] - 46s 3ms/step - loss: 0.0133 - accuracy: 0.9965 - val_loss: 0.0130 - val_accuracy: 0.9965
Epoch 5/5
17576/17576 [==============================] - 44s 3ms/step - loss: 0.0125 - accuracy: 0.9966 - val_loss: 0.0124 - val_accuracy: 0.9966


In [26]:

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

5493/5493 [==============================] - 9s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    174898
           1       0.73      0.46      0.56       859

    accuracy                           1.00    175757
   macro avg       0.86      0.73      0.78    175757
weighted avg       1.00      1.00      1.00    175757

[[174750    148]
 [   467    392]]


**Implementing Convolutional Neural Network (CNN)**

This is a sequential model for a 1D Convolutional Neural Network (CNN) designed for some form of sequence data (possibly time-series data). It consists of the following layers:

Conv1D Layer (64 filters, kernel size 3): This layer performs a 1D convolution operation with 64 filters and a kernel size of 3. It uses ReLU activation.

MaxPooling1D Layer (pool size 2): After the convolution, there's a max-pooling layer with a pool size of 2.

Conv1D Layer (128 filters, kernel size 3): Another convolutional layer with 128 filters and kernel size 3, also using ReLU activation.

MaxPooling1D Layer (pool size 2): Another max-pooling layer with a pool size of 2.

Flatten Layer: This layer flattens the output from the previous layers into a 1D vector, preparing it for the fully connected layers.

Dense Layer (64 units): A fully connected layer with 64 units and ReLU activation.

Dropout Layer: A dropout layer with a dropout rate of 0.5, which helps prevent overfitting.

Dense Layer (1 unit): The output layer with a single unit and sigmoid activation, indicating binary classification.

The model is compiled using the Adam optimizer, binary cross-entropy loss, and accuracy as the metric, just like the first model.

In summary, the first model is a feedforward neural network, while the second model is a 1D convolutional neural network, typically used for sequence or time-series data. Both models are designed for binary classification tasks.

In [27]:
# Define the CNN-based deep learning model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)



Epoch 1/5
17576/17576 [==============================] - 104s 6ms/step - loss: 0.0223 - accuracy: 0.9949 - val_loss: 0.0173 - val_accuracy: 0.9951
Epoch 2/5
17576/17576 [==============================] - 103s 6ms/step - loss: 0.0184 - accuracy: 0.9952 - val_loss: 0.0161 - val_accuracy: 0.9953
Epoch 3/5
17576/17576 [==============================] - 107s 6ms/step - loss: 0.0172 - accuracy: 0.9954 - val_loss: 0.0154 - val_accuracy: 0.9963
Epoch 4/5
17576/17576 [==============================] - 104s 6ms/step - loss: 0.0165 - accuracy: 0.9955 - val_loss: 0.0138 - val_accuracy: 0.9960
Epoch 5/5
17576/17576 [==============================] - 119s 7ms/step - loss: 0.0161 - accuracy: 0.9955 - val_loss: 0.0137 - val_accuracy: 0.9962


In [28]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

5493/5493 [==============================] - 12s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    174898
           1       0.66      0.53      0.59       859

    accuracy                           1.00    175757
   macro avg       0.83      0.76      0.79    175757
weighted avg       1.00      1.00      1.00    175757

[[174666    232]
 [   405    454]]


**Implementing LSTM Model**

LSTM is a type of recurrent neural network (RNN) architecture, specifically designed to handle and capture long-range dependencies and temporal patterns in sequential data. LSTMs are often used in tasks such as time series analysis, natural language processing, and any task where the order of data points is significant.

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
17576/17576 [==============================] - 261s 15ms/step - loss: 0.0325 - accuracy: 0.9948 - val_loss: 0.0238 - val_accuracy: 0.9951
Epoch 2/5
17576/17576 [==============================] - 257s 15ms/step - loss: 0.0202 - accuracy: 0.9950 - val_loss: 0.0175 - val_accuracy: 0.9952
Epoch 3/5
17576/17576 [==============================] - 245s 14ms/step - loss: 0.0171 - accuracy: 0.9952 - val_loss: 

In [30]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

5493/5493 [==============================] - 25s 5ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    174898
           1       0.70      0.53      0.60       859

    accuracy                           1.00    175757
   macro avg       0.85      0.76      0.80    175757
weighted avg       1.00      1.00      1.00    175757

[[174705    193]
 [   405    454]]


**Implementing GRU Model**

A GRU is a type of recurrent neural network (RNN) that is designed to capture and model sequential data. It is similar to the more complex LSTM (Long Short-Term Memory) but is computationally more efficient and can be easier to train.

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model = Sequential()
model.add(GRU(units=64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 64)                12864     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12929 (50.50 KB)
Trainable params: 12929 (50.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
17576/17576 [==============================] - 249s 14ms/step - loss: 0.0210 - accuracy: 0.9951 - val_loss: 0.0169 - val_accuracy: 0.9959
Epoch 2/5
17576/17576 [==============================] - 249s 14ms/step - loss: 0.0159 - accuracy: 0.9961 - val_loss: 0.0146 - val_accuracy: 0.9963
Epoch 3/5
17576/17576 [==============================] - 248s 14ms/step - loss: 0.0151 - accuracy: 0.9962 - val_loss: 

In [32]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

5493/5493 [==============================] - 23s 4ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    174898
           1       0.87      0.38      0.53       859

    accuracy                           1.00    175757
   macro avg       0.93      0.69      0.76    175757
weighted avg       1.00      1.00      1.00    175757

[[174847     51]
 [   531    328]]


**Implementing Auto encoder model**

An autoencoder is a type of neural network used for dimensionality reduction, feature learning, and anomaly detection. It consists of an encoder network that maps the input data to a lower-dimensional representation and a decoder network that attempts to reconstruct the original input from this representation.

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the encoder part of the autoencoder
encoder = Sequential()
encoder.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
encoder.add(Dense(64, activation='relu'))

# Define the decoder part of the autoencoder
decoder = Sequential()
decoder.add(Dense(128, activation='relu', input_shape=(64,)))
decoder.add(Dense(X_train.shape[1], activation='sigmoid'))

# Combine the encoder and decoder to create the autoencoder
autoencoder = Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()
autoencoder.fit(X_train, X_train, epochs=5, batch_size=32, validation_split=0.2)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 64)                11200     
                                                                 
 sequential_5 (Sequential)   (None, 22)                11158     
                                                                 
Total params: 22358 (87.34 KB)
Trainable params: 22358 (87.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
17576/17576 [==============================] - 49s 3ms/step - loss: 0.5794 - val_loss: 0.5827
Epoch 2/5
17576/17576 [==============================] - 47s 3ms/step - loss: 0.5759 - val_loss: 0.5819
Epoch 3/5
17576/17576 [==============================] - 52s 3ms/step - loss: 0.5754 - val_loss: 0.5816
Epoch 4/5
17576/17576 [==============================] - 51s 3ms/step - loss: 0.5752 - val_loss: 0.581

In [34]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

5493/5493 [==============================] - 22s 4ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    174898
           1       0.87      0.38      0.53       859

    accuracy                           1.00    175757
   macro avg       0.93      0.69      0.76    175757
weighted avg       1.00      1.00      1.00    175757

[[174847     51]
 [   531    328]]


In [35]:
pip install seaborn
